In [ ]:
from datetime import timedelta
from rich import print
from dynaconf import Dynaconf
from arango import ArangoClient
from utinni import Context
from pathlib import Path

from tiro.plugins.utinni import TiroTSPump
from tiro.plugins.arango import ArangoAgent
from tiro.core import Scenario
from influxdb_client import InfluxDBClient

conf = Dynaconf(settings_files=["./config/utinni_config.toml"])
scenario = Scenario.from_yaml(Path("./config/scenario1.yaml"), Path("./config/use1.yaml"))

In [ ]:
context = Context(clients=dict(influxdb=InfluxDBClient(**conf.influxdb),
                               arangodb=ArangoAgent(**conf.arangodb)),
                  **conf.on_request)

context.add_pump("tiro", TiroTSPump(scenario=scenario))

In [ ]:
table = context.tiro_table(".*Server%(CPU|Memory)Temperature", column="Server", only_ts=False, fill_with_default=True)

In [ ]:
context.bind(start=-timedelta(hours=1),
             step=timedelta(minutes=10))

# table.value
# table["MemoryTemperature"].value
table["CPUTemperature"].value

In [ ]:
context.tiro_table(pattern=".*Server%(CPU|Memory)Temperature", type="status", as_df=True, include_tags="Server")["CPUTemperature"].value

In [ ]:
print(context.tiro_table(pattern=".*Site", type="status", fill_with_default=True).value)